In [1]:
# install sdk for use gimini
%pip install --upgrade --quiet google-cloud-aiplatform

In [2]:
# import library
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

In [3]:
# get project by CLI
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-east1"
# start vertex
vertexai.init(project=PROJECT_ID, location=REGION)

In [4]:
# select the model
model = GenerativeModel("gemini-pro")

In [5]:
creative_gen_config = GenerationConfig(temperature=1, top_p=0.8)

In [6]:
prompt = "Give some ideas for using LLM for create startup?"

response = model.generate_content(prompt,
                                  generation_config=creative_gen_config)
print(response.text)

##  Ideas for Using LLMs in Startup Creation

LLMs are powerful tools with the potential to disrupt various industries, including the startup ecosystem. Here are some ideas for how you can leverage LLMs to create a successful startup:

**Product Development:**

* **Develop AI-powered solutions:** LLMs can be used to create chatbots, virtual assistants, and other AI-powered solutions for various industries. 
* **Generate creative content:** LLMs can generate text, code, and even images, which can be used to create marketing materials, product descriptions, and even design prototypes.
* **Personalize user experiences:** LLMs can analyze user data and provide personalized recommendations, content, and experiences.
* **Automate tasks:** LLMs can automate repetitive tasks, such as data entry, customer service, and content creation.

**Market Research and Analysis:**

* **Analyze market trends:** LLMs can analyze large amounts of data to identify market trends and opportunities.
* **Target p

In [ ]:
prompt = "Provide ten interview questions for the role of prompt engineer."

response = model.generate_content(prompt,
                                  generation_config=creative_gen_config)
print(response.text)

In [7]:
predictable_gen_config = GenerationConfig(temperature=0.1, top_p=0.1)

In [8]:
prompt = """
Given a user's input, classify their intent, such as "finding information", "making a reservation", or "placing an order". \n
user input: Hi, can you please book a table for two at Juan for May 1?
"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

## Intent: Making a reservation

The user is requesting to book a table for two at a restaurant called Juan for May 1st. 



In [9]:
prompt = """
What is the topic for a given news headline? \n
- business \n
- entertainment \n
- health \n
- sports \n
- technology \n\n

Text: Pixel 7 Pro Expert Hands On Review. \n
The answer is: technology \n

Text: Quit smoking? \n
The answer is: health \n

Text: Birdies or bogeys? Top 5 tips to hit under par \n
The answer is: sports \n

Text: Relief from local minimum-wage hike looking more remote \n
The answer is: business \n

Text: You won't guess who just arrived in Bari, Italy for the movie premiere. \n
The answer is:
"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

entertainment 



In [10]:
import pandas as pd
from sklearn.metrics import classification_report

review_data_df = pd.DataFrame(
    {
        "review": [
            "i love this product. it does have everything i am looking for!",
            "all i can say is that you will be happy after buying this product",
            "its way too expensive and not worth the price",
            "i am feeling okay. its neither good nor too bad.",
        ],
        "sentiment_groundtruth": ["positive", "positive", "negative", "neutral"],
    }
)

review_data_df

,review,sentiment_groundtruth
0,i love this product. it does have everything i...,positive
1,all i can say is that you will be happy after ...,positive
2,its way too expensive and not worth the price,negative
3,i am feeling okay. its neither good nor too bad.,neutral


In [12]:
def get_sentiment(row):
    prompt = f"""Classify the sentiment of the following review as "positive", "neutral" or "negative". Return only the classification.
                review: {row}
              """
    response = model.generate_content(
        contents=prompt, generation_config=predictable_gen_config
    ).text
    return response


review_data_df["sentiment_prediction"] = review_data_df["review"].apply(get_sentiment)
review_data_df

,review,sentiment_groundtruth,sentiment_prediction
0,i love this product. it does have everything i...,positive,positive
1,all i can say is that you will be happy after ...,positive,positive
2,its way too expensive and not worth the price,negative,negative
3,i am feeling okay. its neither good nor too bad.,neutral,neutral


In [13]:
report = classification_report(
    review_data_df["sentiment_groundtruth"], review_data_df["sentiment_prediction"]
)
Markdown(report)

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00         1
     neutral       1.00      1.00      1.00         1
    positive       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4


In [14]:
predictable_gen_config = GenerationConfig(temperature=0.1, top_p=0.1)

In [16]:
text = """Alphabet’s (GOOG, GOOGL) Google unit is developing artificial intelligence technology that can take over a user’s internet browser, The Information reported Saturday, citing three people with direct knowledge of the product.

According to the report, the technology – named Project Jarvis – would be able to complete tasks such as internet searches, shopping, and booking travel, the people were quoted as saying. """

In [17]:
# text summary
prompt = f"""
Provide a very short summary, no more than three sentences, for the following article: {text}

"""
response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

Google is developing an AI technology called Project Jarvis that can take over a user's internet browser and complete tasks such as internet searches, shopping, and booking travel. The technology is still in development and is not yet available to the public. 



In [19]:
#text extraction
order = "We need eight grilled cheese sandwiches. Two with swiss cheese, three with muenster, three with cheddar."

prompt = """
    Break the customer's order into individual items with keys for the following fields:
    - item_name
    - cheese_selection

    Order:
    {order_field}
""".format(order_field=order)

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
print(response.text)

```json
[
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Swiss"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Swiss"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  }
]
```


In [20]:
prompt = """Q: Who was President of the United States in 1955?
              Which party did he belong to?
            A:
        """
response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
print(response.text)

Dwight D. Eisenhower was President of the United States in 1955. He belonged to the Republican Party.


In [21]:
context = """
Storage and content policy \n
How durable is my data in Cloud Storage? \n
Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
"""

question = "How is high availability achieved?"

prompt = f"""Answer the question given in the contex below:
Context: {context}?\n
Question: {question} \n
Answer:
"""

print("[Prompt]")
print(prompt)

print("[Response]")
response = model.generate_content(contents=prompt, generation_config=predictable_gen_config)
print(response.text)

[Prompt]
Answer the question given in the contex below:
Context: 
Storage and content policy 

How durable is my data in Cloud Storage? 

Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
?

Question: How is high availability achieved? 

Answer:

[R